[View in Colaboratory](https://colab.research.google.com/github/aminzabardast/Tensorflow-Tutorials/blob/dev/5_Convolutional_Neural_Network.ipynb)

# Convolutional Neural Networks With Tensor Flow

In [1]:
!pip show tensorflow
!rm -rf logs

Name: tensorflow
Version: 1.9.0rc1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: grpcio, numpy, wheel, gast, setuptools, termcolor, absl-py, tensorboard, protobuf, six, astor
Required-by: 


In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

def compute_accuracy(v_xs, v_ys):
    y_pre = sess.run(rfc2, feed_dict={xs: v_xs, dop: 1})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, dop: 1})
    return result

def weights_variable(shape):
    initial = tf.truncated_normal(shape, stddev=.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, name=None):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME', name=name)

def maxPool2x2(x, name=None):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=name)  # Squeezing each 2x2 box of pixels into 1 pixel and keeping the max value

# Input
with tf.name_scope('Input'):
    xs = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='Data')/255
    x_images = tf.reshape(xs,shape=[-1, 28, 28, 1], name='ReshapedData')
    ys = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='Truth')
with tf.name_scope('OtherVariable'):
    dop = tf.placeholder(dtype=tf.float32, name='DropOutProb')

# Conv 1
with tf.name_scope('Conv1'):
    kernel1 = weights_variable([5,5,1,32])  # 5x5 Kernel, Feature Map Input 1 (Gray scale image) and Output 32
    bconv1 = bias_variable([32])
    rconv1 = tf.nn.relu(conv2d(x_images, kernel1) + bconv1, name='Output')  # Using kernel1 to calculate the convolved layer / Output size: 28 x 28 x 32

# Pool 1
with tf.name_scope('MaxPool1'):
    rpool1 = maxPool2x2(rconv1, name='Output')  # Max polling the result of Conv1 layer / Output size: 14 x 14 x 32

# Conv 2
with tf.name_scope('Conv2'):
    kernel2 = weights_variable([5, 5, 32, 64])  # 5x5 Kernel, Feature Map Input 32 and Output 64
    bconv2 = bias_variable([64])
    rconv2 = tf.nn.relu(conv2d(rpool1, kernel2) + bconv2, name='Output')  # Using kernel2 to calculate the convolved layer / Output size: 14 x 14 x 64

# Pool 2
with tf.name_scope('Pool2'):
    rpool2 = maxPool2x2(rconv2, name='Output')  # Max polling the result of Conv2 layer / Output size: 7 x 7 x 64

# FC 1
with tf.name_scope('FC1'):
    weights1 = weights_variable([7*7*64, 1024])  # Weights
    biases1 = bias_variable([1024])  # Biases
    rpool2flat = tf.reshape(rpool2, shape=[-1, 7*7*64])  # From [n,7,7,64] to [n,3136]
    rfc1 = tf.nn.relu(tf.matmul(rpool2flat, weights1) + biases1, name='Output')

# Dropout
with tf.name_scope('DropOut'):
    rdo = tf.nn.dropout(rfc1, dop)

# FC 2
with tf.name_scope('FC2_CLS'):
    weights2 = weights_variable([1024, 10])
    biases2 = bias_variable([10])
    rfc2 = tf.nn.softmax(tf.matmul(rdo, weights2) + biases2, name='Output')

# Loss function
with tf.name_scope('Loss'):
    crossEntropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(rfc2), reduction_indices=[1]))
    tf.summary.scalar(name='CrossEntropy', tensor=crossEntropy)

# Optimizer
with tf.name_scope('Optimizer'):
    trainStep = tf.train.AdamOptimizer().minimize(crossEntropy)

init = tf.global_variables_initializer()
sess = tf.Session()
merged = tf.summary.merge_all()
trainWriter = tf.summary.FileWriter(logdir='logs/train', graph=sess.graph)
testWriter = tf.summary.FileWriter(logdir='logs/test', graph=sess.graph)
sess.run(init)


for epoch in range(1000):
    trainBatchXs, trainBatchYs = mnist.train.next_batch(100)
    testBatchXs, testBatchYs = mnist.test.next_batch(100)
    sess.run(trainStep, feed_dict={xs: trainBatchXs, ys: trainBatchYs, dop: 1})
    trainWriter.add_summary(sess.run(merged, feed_dict={xs: trainBatchXs, ys: trainBatchYs, dop: 1}), epoch)
    testWriter.add_summary(sess.run(merged, feed_dict={xs: testBatchXs, ys: testBatchYs, dop: 1}), epoch)
    if epoch % 100 == 0:
        print(epoch)

print(compute_accuracy(mnist.test.images, mnist.test.labels))


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
0
100
200
300
400
500
600
700
800
900
0.9873


In [3]:
!tar czvf logs.tar.gz logs

from google.colab import files
files.download('logs.tar.gz')

!rm -rvf logs*

logs/
logs/test/
logs/test/events.out.tfevents.1530466669.549f4ad22cd1
logs/train/
logs/train/events.out.tfevents.1530466669.549f4ad22cd1
removed 'logs/test/events.out.tfevents.1530466669.549f4ad22cd1'
removed directory 'logs/test'
removed 'logs/train/events.out.tfevents.1530466669.549f4ad22cd1'
removed directory 'logs/train'
removed directory 'logs'
removed 'logs.tar.gz'
